In [1]:
import pandas as pd
import numpy as np
import pyodbc
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta
from datetime import date
import tkinter as tk
from tkinter.filedialog import askopenfilename
import sqlite3

In [36]:
conn = sqlite3.connect('../../../InSyncConnection/Database/InSyncClinical.db')

## Notes
- Transfer notes are not always being picked up properly.
- Discharge notes are not reseting the process
- Make sure that due dates are not calculated by time
- If the are two initial treatment plans go by the second one.

In [3]:
sql='''
SELECT 
    ptPatient_Activity.*,
    MRNNumber,
    LastName || ', ' || FirstName as patient_fullName,
    DOB,
    FacilityName AS 'Primary Facility'
FROM 
    ptPatient_Activity
    LEFT JOIN emr_PatientDetails ON (ptPatient_Activity.PatientID=emr_PatientDetails.PatientID)
    LEFT JOIN emr_FacilitiesDetails ON (emr_PatientDetails._PrimaryFacility = emr_FacilitiesDetails.FacilityId)
'''
patientDetails = pd.read_sql(sql, conn)
patientDetails = patientDetails.replace("", np.nan)
patientDetails['DOB'] =pd.to_datetime(patientDetails['DOB'])

In [4]:
sql='''
SELECT 
    tblEncounterSummary.PatientId as PatientID,
    tblEncounterSummary.EncounterID, 
    tblResourceDetails.FullName as ProviderName,
    tblEncounter.EndDate,
    VisitDateTime as EncounterDate,
    EncounterEndDateTimeToolTip,
    EncounterStartDateTimeToolTip,
    FacilityName,
    tblEncounterType.EncounterType,
    EncounterOpenClosed,
    ProgramName,
    closedByLastName || ', ' || closedByFirstName AS closedByFullName
FROM 
    tblEncounterSummary
    LEFT JOIN emr_FacilitiesDetails ON (tblEncounterSummary.FacilityId = emr_FacilitiesDetails.FacilityId)
    LEFT JOIN tblEncounterEndDetails ON (tblEncounterSummary.EncounterID = tblEncounterEndDetails.EncounterID)
    INNER JOIN tblEncounter ON (tblEncounterSummary.EncounterID = tblEncounter.EncounterID)
    LEFT JOIN tblEncounterType ON (tblEncounterSummary.EncounterTypeID = tblEncounterType.EncounterTypeID)
    LEFT JOIN tblResourceDetails ON (tblEncounterSummary.ProviderId = tblResourceDetails.ResourceId)
WHERE (tblEncounterType.EncounterType LIKE "%treatment%"
    OR tblEncounterType.EncounterType LIKE "%transfer%"
    OR tblEncounterType.EncounterType LIKE "%Initial Eval%"
    OR tblEncounterType.EncounterType LIKE "%discharge%")
    AND tblEncounterType.EncounterType NOT LIKE "%CFTSS%"
'''
treatmentPlanEncounters = pd.read_sql(sql, conn)
treatmentPlanEncounters = treatmentPlanEncounters.replace("", np.nan)
treatmentPlanEncounters['EncounterDate'] = pd.to_datetime(treatmentPlanEncounters['EncounterDate'])
treatmentPlanEncounters['EndDate'] = pd.to_datetime(treatmentPlanEncounters['EndDate'])
# Remove the word time: from column
treatmentPlanEncounters['EncounterStartDateTimeToolTip'] = treatmentPlanEncounters['EncounterStartDateTimeToolTip'].str.replace("Time: ", "")
treatmentPlanEncounters['EncounterEndDateTimeToolTip'] = treatmentPlanEncounters['EncounterEndDateTimeToolTip'].str.replace("Time: ", "")

treatmentPlanEncounters['EncounterStartDateTimeToolTip'] = pd.to_datetime(treatmentPlanEncounters['EncounterStartDateTimeToolTip'])
treatmentPlanEncounters['EncounterEndDateTimeToolTip'] = pd.to_datetime(treatmentPlanEncounters['EncounterEndDateTimeToolTip'])

In [5]:
sql = f'''
SELECT
PatientId as PatientID
FROM 
    tblEncounterSummary
LEFT JOIN
    tblEncounterType ON tblEncounterType.EncounterTypeID = tblEncounterSummary.EncounterTypeID
WHERE 
       tblEncounterType.EncounterType LIKE "%Psychotherapy%" 
       OR 
       (tblEncounterType.EncounterType LIKE "%Initial Eval%" 
       AND tblEncounterType.EncounterType LIKE "%2%" )
'''
acceptedPatients = pd.read_sql(sql,conn)
acceptedPatients.drop_duplicates('PatientID', inplace=True)
acceptedPatients['Admitted']="True"
treatmentPlanEncounters= treatmentPlanEncounters.merge(acceptedPatients, on='PatientID', how='left')
treatmentPlanEncounters['Admitted'].fillna("False", inplace=True)

In [6]:
def find_missing_initial_treatment_plan(temp_df):
    if "Initial Treatment Plan" not in temp_df['EncounterType'].unique():
        return True
    else:
        return False
    
def getDischargeInfo(is_ActiveInInSync, is_lastEncounterDischarge):
    if is_ActiveInInSync and is_lastEncounterDischarge:
        return """Last encounter was discharge so Patient should probably be inactivated | """    

def filterAndSortToRelevantData(completeDataFrame, patientID):
    singlePatient_df = completeDataFrame[completeDataFrame['PatientID'] == patientID].copy()
    singlePatient_df = singlePatient_df[singlePatient_df['Summarized Encounter Type'].map(lambda encType: True if encType == 'Treatment Plan' or encType == 'Transfer Note' or encType == 'Discharge Note' else False)].copy()
    singlePatient_df = singlePatient_df.sort_values(by="Review Date", ascending=True)
    singlePatient_df = singlePatient_df.reset_index(drop=True)
    return singlePatient_df


def isEmptyDataFrame(singlePatient_df):     
    if singlePatient_df.shape[0] == 0:
        return True
    else:
        return False
    
def isNeedingToBeInactivated(is_ActiveInInSync, is_lastEncounterDischarge):
    if is_ActiveInInSync and is_lastEncounterDischarge:
        return True  
    else:
        return False

def isTransferNote(singlePatient_df, currentEncounterIndexNum):
    if singlePatient_df[currentEncounterIndexNum, 'Summarized Encounter Type'] == 'Transfer Note':
        return True
    else:
        return False

def hasNextPlan(singlePatient_df, currentEncounterIndexNum):
    # and see if there are any treatment plans after current treatment plan
    currentTreatmentPlanDate = singlePatient_df.loc[currentEncounterIndexNum, 'EncounterDate']
    if 'Treatment Plan' in singlePatient_df[singlePatient_df['EncounterDate'] > currentTreatmentPlanDate]["Summarized Encounter Type"].unique():
        return True
    else:
        return False

def getLocationOfNextPlan(singlePatient_df, currentEncounterIndexNum):
    currentTreatmentPlanDate = singlePatient_df.loc[currentEncounterIndexNum, 'EncounterDate']
    nextPlanLocation = singlePatient_df[(singlePatient_df['EncounterDate'] > currentTreatmentPlanDate) & (singlePatient_df["Summarized Encounter Type"] == 'Treatment Plan') ].index[0]
    return nextPlanLocation

def isTransferedNext(singlePatient_df, currentEncounterIndexNum, hasNextPlan, nextPlanLocation=None):
    currentTreatmentPlanDate = singlePatient_df.loc[currentEncounterIndexNum, 'EncounterDate']
    if hasNextPlan:
        # see if there are any transfer notes between current plan and next plan
        nextTreatmentPlanDate =  singlePatient_df.loc[nextPlanLocation, 'EncounterDate']
        if 'Transfer Note' in singlePatient_df[(singlePatient_df['EncounterDate'] > currentTreatmentPlanDate) & ((singlePatient_df['EncounterDate'] < nextTreatmentPlanDate))]["Summarized Encounter Type"].unique():
            return True
        else:
            return False
    else:
        if 'Transfer Note' in singlePatient_df[singlePatient_df['EncounterDate'] >= currentTreatmentPlanDate]["Summarized Encounter Type"].unique():
            return True
        else:
            return False
    
def isDischargedNext(singlePatient_df, currentEncounterIndexNum, hasNextPlan, nextPlanLocation=None):
    currentTreatmentPlanDate = singlePatient_df.loc[currentEncounterIndexNum, 'EncounterDate']
    if hasNextPlan: # see if there are any transfer notes between current plan and next plan
        nextTreatmentPlanDate =  singlePatient_df.loc[nextPlanLocation, 'EncounterDate']
        if 'Discharge Note' in singlePatient_df[(singlePatient_df['EncounterDate'] > currentTreatmentPlanDate) & 
                                           ((singlePatient_df['EncounterDate'] < nextTreatmentPlanDate))]["Summarized Encounter Type"].unique():
            return True
        else:
            return False
    else: # see if there is a 
        if 'Discharge Note' in singlePatient_df[singlePatient_df['EncounterDate'] >= currentTreatmentPlanDate]["Summarized Encounter Type"].unique():
            return True
        else:
            return False    
    
def isNextPlanClosedLate(singlePatient_df, currentEncounterIndexNum, hasNextPlan, nextPlanLocation):
    if singlePatient_df.loc[nextPlanLocation, "EndDate"]!= pd.NaT and singlePatient_df.loc[nextPlanLocation, "EndDate"] > singlePatient_df.loc[currentEncounterIndexNum, "Review Date"]:
        return True
    else:
        return False

def isCurrentPlanSigned(singlePatient_df, currentEncounterIndexNum):
    if str(singlePatient_df.loc[currentEncounterIndexNum, "closedByFullName"]) =='None':
        return False
    else:
        return True

def getDaysAfterTransfer(singlePatient_df, currentEncounterIndexNum, hasNextPlan, nextPlanLocation=None):
    currentTreatmentPlanDate = singlePatient_df.loc[currentEncounterIndexNum, 'EncounterDate']
    if hasNextPlan:
        nextTreatmentPlanDate =  singlePatient_df.loc[nextPlanLocation, 'EncounterDate']
        return (nextTreatmentPlanDate-singlePatient_df.loc[currentEncounterIndexNum, "EncounterDate"]).days
    else:
        daysAfterTransfer = (datetime.now()-currentTreatmentPlanDate).days
        return daysAfterTransfer

In [7]:
relevantInfodf = treatmentPlanEncounters.merge(patientDetails, how='left', on='PatientID')
relevantInfodf = relevantInfodf[relevantInfodf['MRNNumber'].notnull()].copy()

In [8]:
relevantInfodf['Summarized Encounter Type'] = relevantInfodf['EncounterType'].map(lambda type: 
                                                          "Treatment Plan" if "treatment" in type.lower() else 
                                                           ("Initial Eval" if 'evaluation' in type.lower() else 
                                                            ("Transfer Note" if 'transfer' in type.lower() else 
                                                             ("Discharge Note" if 'discharge' in type.lower() else "Something else"))))
relevantInfodf['Review Date'] = relevantInfodf.apply(lambda row: row['EncounterDate'] + relativedelta(months=+11) if row['EncounterType'] == 'Initial Treatment Plan' else row['EncounterDate'] + relativedelta(months=+12), axis=1)
relevantInfodf['Days to Review Date'] = relevantInfodf['Review Date'].map(lambda reviewDate:  reviewDate.date()) - date.today()


In [9]:
allPatientDf = pd.DataFrame()
for patientID in relevantInfodf['PatientID'].unique()[0:]:
    singlePatient_df = filterAndSortToRelevantData(completeDataFrame=relevantInfodf, patientID=patientID)
    if isEmptyDataFrame(singlePatient_df): #if patient has initial eval with no initial treatment plan
        emptySinglePatient_df = relevantInfodf[(relevantInfodf['PatientID'] == patientID)].copy()
        
        missingInitialTreatmentPlan = f"Missing Initial Treatment Plan"
        emptySinglePatient_df["missingInitialTreatmentPlan"] = True
        emptySinglePatient_df["treatmentPlanStatus"] = missingInitialTreatmentPlan

        allPatientDf = pd.concat([allPatientDf,emptySinglePatient_df])
        continue
    else:
        patientOverallStatus = ""
        
        missingInitialTreatmentPlan = find_missing_initial_treatment_plan(singlePatient_df)
        singlePatient_df['missingInitialTreatmentPlan'] = missingInitialTreatmentPlan        
        singlePatient_df["treatmentPlanStatus"] = missingInitialTreatmentPlan
        
        is_ActiveInInSync = singlePatient_df.loc[0, 'is_ActiveInInSync']
        is_lastEncounterDischarge = singlePatient_df.loc[0, 'is_lastEncounterDischarge']
        
        is_NeedToBeInactivated = isNeedingToBeInactivated(is_ActiveInInSync, is_lastEncounterDischarge)
        singlePatient_df['is_NeedToBeInactivated'] = is_NeedToBeInactivated
        
        for currentEncounterIndexNum in range(singlePatient_df.shape[0]):
            treatmentPlanStatus = patientOverallStatus
            currentEncounterType = singlePatient_df.loc[currentEncounterIndexNum, 'Summarized Encounter Type']
            daysToReviewDate = singlePatient_df.loc[currentEncounterIndexNum, 'Days to Review Date'].days
            
            has_NextPlan = hasNextPlan(singlePatient_df, currentEncounterIndexNum)
            singlePatient_df.loc[currentEncounterIndexNum, "has_NextPlan"] = has_NextPlan
                        
            is_CurrentPlanSigned = isCurrentPlanSigned(singlePatient_df, currentEncounterIndexNum)
            singlePatient_df.loc[currentEncounterIndexNum, "is_CurrentPlanSigned"] = is_CurrentPlanSigned
            
            if not is_CurrentPlanSigned:
                treatmentPlanStatus += "Plan Still needs to be signed | "       
                
            if is_NeedToBeInactivated:
                treatmentPlanStatus += "Last encounter was discharge so Patient should probably be inactivated | "
            if has_NextPlan:
                next_PlanLocation = getLocationOfNextPlan(singlePatient_df, currentEncounterIndexNum)
                singlePatient_df.loc[currentEncounterIndexNum, "next_PlanLocation"] = next_PlanLocation
                
                is_TransferedNext = isTransferedNext(singlePatient_df, currentEncounterIndexNum, has_NextPlan, next_PlanLocation)
                singlePatient_df.loc[currentEncounterIndexNum, "is_TransferedNext"] = is_TransferedNext
                
                is_DischargedNext = isDischargedNext(singlePatient_df, currentEncounterIndexNum, has_NextPlan, next_PlanLocation)
                singlePatient_df.loc[currentEncounterIndexNum, "is_DischargedNext"] = is_DischargedNext
                if (is_TransferedNext and not is_DischargedNext) and is_CurrentPlanSigned and has_NextPlan:
                    get_DaysAfterTransfer = getDaysAfterTransfer(singlePatient_df, currentEncounterIndexNum, has_NextPlan, next_PlanLocation)
                    treatmentPlanStatus += f"Old Plan is Good | Patient was Transfered and they completed the follow up plan {get_DaysAfterTransfer} days after."
                elif (is_DischargedNext and not is_TransferedNext) and is_CurrentPlanSigned and has_NextPlan:
                    treatmentPlanStatus += f"Patient was Discharged"
                elif not is_TransferedNext and not is_DischargedNext and is_CurrentPlanSigned and has_NextPlan:
                    treatmentPlanStatus += "Everything Seems Good"
                
                is_NextPlanClosedLate = isNextPlanClosedLate(singlePatient_df, currentEncounterIndexNum, has_NextPlan, next_PlanLocation)
                singlePatient_df.loc[currentEncounterIndexNum, "is_NextPlanClosedLate"] = is_NextPlanClosedLate
                
            else: # if there is no next plan 
                is_TransferedNext = isTransferedNext(singlePatient_df, currentEncounterIndexNum, has_NextPlan)
                singlePatient_df.loc[currentEncounterIndexNum, "is_TransferedNext"] = is_TransferedNext
                
                is_DischargedNext = isDischargedNext(singlePatient_df, currentEncounterIndexNum, has_NextPlan)
                singlePatient_df.loc[currentEncounterIndexNum, "is_DischargedNext"] = is_DischargedNext
                
                if not is_TransferedNext and not is_DischargedNext and daysToReviewDate > 0:
                    treatmentPlanStatus += f"Everything Seems Good. Still have {daysToReviewDate} days left."
                elif not is_TransferedNext and not is_DischargedNext and daysToReviewDate < 0:
                    treatmentPlanStatus += f"Plan is {np.absolute(daysToReviewDate)} days overdue."
                elif is_TransferedNext and not is_DischargedNext:
                    daysAfterTransfer = getDaysAfterTransfer(singlePatient_df, currentEncounterIndexNum, has_NextPlan)
                    if daysAfterTransfer > 30 and currentEncounterType == "Transfer Note":
                        treatmentPlanStatus += f"Patient Transfered. Plan is {daysAfterTransfer-30} days overdue."
                    elif daysAfterTransfer > 30:
                        treatmentPlanStatus += f"Patient Transfered."
                    else:
                        treatmentPlanStatus += f"Patient Transfered. Plan is due in {30 - daysAfterTransfer} days."
                elif (is_DischargedNext and not is_TransferedNext):
                    treatmentPlanStatus += f"Patient was Discharged"   
            print(treatmentPlanStatus)
            singlePatient_df.loc[currentEncounterIndexNum, "treatmentPlanStatus"] = treatmentPlanStatus  
        allPatientDf = pd.concat([allPatientDf,singlePatient_df])

Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Everything Seems Good
Plan Still needs to be signed | Patient Transfered.
Patient Transfered. Plan is 79 days overdue.
Everything Seems Good
Everything Seems Good
Everything Seems Good
Everything Seems Good
Everything Seems Good. Still have 227 days left.
Plan Still needs to be signed | Everything Seems Good. Still have 230 days left.
Patient was Discharged
Patient was Discharged
Everything Seems Good
Everything Seems Good
Everything Seems Good
Patient Transfered.
Patient Transfered. Plan is 2 days overdue.
Everything Seems Good
Everything Seems Good. Still have 30 days left.
Last encounter was discharge so Patient should probably be inactivated | Everything Seems Good
Plan Still needs to be signed | Last encounter was discharge so Patient should probably be inactivated | 
Last encounter was disch

C:\Users\ylen\AppData\Local\Temp\ipykernel_14508\2586010412.py:11: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  allPatientDf = pd.concat([allPatientDf,emptySinglePatient_df])


Plan Still needs to be signed | Everything Seems Good. Still have 10 days left.
Everything Seems Good
Everything Seems Good. Still have 234 days left.
Everything Seems Good. Still have 30 days left.
Everything Seems Good. Still have 57 days left.
Plan Still needs to be signed | 
Everything Seems Good. Still have 78 days left.
Everything Seems Good. Still have 44 days left.
Plan Still needs to be signed | 
Everything Seems Good
Everything Seems Good. Still have 27 days left.
Plan is 335 days overdue.
Plan Still needs to be signed | Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Everything Seems Good. Still have 70 days left.
Patient was Discharged
Patient was Discharged
Plan is 342 days overdue.
Patient was Discharged
Patient was Discharged
Patient was Discharged
Old Plan is Good | Patient was Transfered and they completed the follow up plan 32

Patient Transfered.
Patient Transfered.
Patient Transfered. Plan is 242 days overdue.
Everything Seems Good. Still have 156 days left.
Everything Seems Good. Still have 156 days left.
Everything Seems Good. Still have 135 days left.
Everything Seems Good. Still have 82 days left.
Plan Still needs to be signed | Everything Seems Good. Still have 26 days left.
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Everything Seems Good. Still have 113 days left.
Everything Seems Good. Still have 113 days left.
Everything Seems Good. Still have 162 days left.
Everything Seems Good. Still have 72 days left.
Everything Seems Good. Still have 72 days left.
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Last encount

Everything Seems Good
Plan Still needs to be signed | 
Plan Still needs to be signed | Everything Seems Good. Still have 162 days left.
Everything Seems Good
Everything Seems Good. Still have 232 days left.
Last encounter was discharge so Patient should probably be inactivated | Everything Seems Good
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Plan Still needs to be signed | Last encounter was discharge so Patient should probably be inactivated | 
Last encounter was discharge so Patient should probably be inactivated | Everything Seems Good
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Everything Seems Good
Last encounter was

...

Patient was Discharged
Patient was Discharged
Patient was Discharged
Patient was Discharged
Patient was Discharged
Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Everything Seems Good
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged

Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Patient was Discharged
Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Everything Seems Good
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient was Discha

Everything Seems Good
Everything Seems Good. Still have 183 days left.
Everything Seems Good
Everything Seems Good. Still have 183 days left.
Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Everything Seems Good
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Everything Seems Good
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Everything Seems Good. Still have 155 days left.
Everything Seems Goo

Patient was Discharged
Patient was Discharged
Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Patient was Discharged
Patient was Discharged
Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Patient was Discharged
Patient was Discharged
Patient was Discharged
Patient was Discharged
Patient was Discharged
Patient was Discharged
Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Patient was Discharged
Patient was Discharged
Patient was Discharged
Patient was Discharged
Patient was Discharged
Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient was Discharged
Last encounter was discharge so Patient should probably be inactivated | Patient wa

Everything Seems Good. Still have 102 days left.
Plan is 161 days overdue.
Everything Seems Good. Still have 86 days left.
Everything Seems Good
Everything Seems Good
Everything Seems Good
Everything Seems Good. Still have 244 days left.
Old Plan is Good | Patient was Transfered and they completed the follow up plan 83 days after.
Everything Seems Good
Everything Seems Good. Still have 262 days left.
Everything Seems Good. Still have 259 days left.
Everything Seems Good
Plan is 86 days overdue.
Plan is 86 days overdue.
Everything Seems Good
Everything Seems Good
Everything Seems Good
Everything Seems Good
Everything Seems Good
Everything Seems Good
Everything Seems Good. Still have 327 days left.
Everything Seems Good
Everything Seems Good
Everything Seems Good
Everything Seems Good
Everything Seems Good. Still have 313 days left.
Everything Seems Good
Everything Seems Good
Everything Seems Good
Everything Seems Good. Still have 234 days left.
Everything Seems Good
Everything Seems Goo

In [10]:
allPatientDf[["has_NextPlan", "is_CurrentPlanSigned","next_PlanLocation","is_TransferedNext","is_DischargedNext","is_NextPlanClosedLate"]] = allPatientDf[["has_NextPlan", "is_CurrentPlanSigned","next_PlanLocation","is_TransferedNext","is_DischargedNext","is_NextPlanClosedLate"]].fillna(False)   

In [11]:
list(allPatientDf['treatmentPlanStatus'])[2]

'Everything Seems Good'

In [12]:
# for column in allPatientDf.columns:
#     print(column)

In [13]:
allPatientDf['is_TreatmentPlanGood'] = allPatientDf['treatmentPlanStatus'].map(lambda status: True if 'Everything Seems Good' in str(status) else False)
allPatientDf['is_TreatmentPlanOverdue'] = allPatientDf['treatmentPlanStatus'].map(lambda status: True if 'overdue' in str(status).lower() else False)

In [34]:
allPatientDf['treatmentPlanStatus'] = allPatientDf['treatmentPlanStatus'] + "| " + allPatientDf['seen_Recently']

In [15]:
# final_df[final_df['PatientID']==622214]
# final_df[final_df['PatientID']==622741]

In [16]:
allPatientDf['seen_Recently'].unique()

array(["Hasn't been seen in 6 months",
       "They've been seen in the past 3 months",
       "Hasn't been seen in 3 months", None], dtype=object)

In [17]:
# final_df[final_df['allTheInfoYouEverWanted'].map(lambda x: True if "Closed After Due Date" in x else False) ].groupby('ProviderName')['EncounterId'].count().sort_values(ascending=False).head(20)

In [18]:
# final_df[final_df['allTheInfoYouEverWanted']==""]

In [19]:
# Change to integer so SQL can recognize it
allPatientDf['Days to Review Date'] = allPatientDf['Days to Review Date'].dt.days

In [20]:
# filter to useful columns
allPatientDf = allPatientDf[["EncounterId",
                            "MRNNumber",
                            "DOB",
                            "PatientID",
                            "ProviderName",
                            "patient_fullName",
                            "FacilityName",
                            "Summarized Encounter Type",
                            "EncounterDate",
                            "EncounterEndDateTimeToolTip",
                            "EncounterStartDateTimeToolTip",
                            "EncounterType",
                            "EncounterOpenClosed",
                            "ProgramName",
                            "closedByFullName",
                            "finalEncounterType",
                            "finalEncounterDate",
                            "is_ActiveInInSync",
                            "is_lastEncounterDischarge",
                            "lastBillable_EncounterType",
                            "lastBillable_EncounterDate",
                            "lastBillable_ProgramName",
                            "lastBillable_FacilityName",
                            "seen_Recently",
                            "Review Date",
                            "Days to Review Date",
                            "is_NeedToBeInactivated",
                            "has_NextPlan",
                            "is_CurrentPlanSigned",
                            "next_PlanLocation",
                            "is_TransferedNext",
                            "is_DischargedNext",
                            "is_NextPlanClosedLate",
                            "treatmentPlanStatus",
                            "missingInitialTreatmentPlan",
                            "is_TreatmentPlanGood",
                            "is_TreatmentPlanOverdue",
                            "Admitted"]]

## For Presenting
- Replace all names with random ones for presenting

In [21]:
# randomNames = pd.read_excel('../data/Random Names.xlsx')

# final_df['patient_fullName'] = final_df['patient_fullName'].map(lambda name: list(randomNames['Names'].sample(1))[0])
# final_df['ProviderName'] = final_df['ProviderName'].map(lambda name: list(final_df['ProviderName'].sample(1))[0])

In [22]:
# use to transform data into
query = ", ".join([item.replace(" ", "_") + " " + str(allPatientDf[item].dtype) for item in allPatientDf.columns])
query = query.replace("object", "TEXT")
query = query.replace("datetime64[ns]", "TEXT")
query = query.replace("timedelta64[ns]", "TEXT")
query = query.replace("float64", "INTEGER")
query = query.replace("int64", "INTEGER")
query = query.replace("bool", "TEXT")
query

'EncounterId INTEGER, MRNNumber TEXT, DOB TEXT, PatientID INTEGER, ProviderName TEXT, patient_fullName TEXT, FacilityName TEXT, Summarized_Encounter_Type TEXT, EncounterDate TEXT, EncounterEndDateTimeToolTip TEXT, EncounterStartDateTimeToolTip TEXT, EncounterType TEXT, EncounterOpenClosed TEXT, ProgramName TEXT, closedByFullName TEXT, finalEncounterType TEXT, finalEncounterDate TEXT, is_ActiveInInSync INTEGER, is_lastEncounterDischarge INTEGER, lastBillable_EncounterType TEXT, lastBillable_EncounterDate TEXT, lastBillable_ProgramName TEXT, lastBillable_FacilityName TEXT, seen_Recently TEXT, Review_Date TEXT, Days_to_Review_Date INTEGER, is_NeedToBeInactivated TEXT, has_NextPlan TEXT, is_CurrentPlanSigned TEXT, next_PlanLocation TEXT, is_TransferedNext TEXT, is_DischargedNext TEXT, is_NextPlanClosedLate TEXT, treatmentPlanStatus TEXT, missingInitialTreatmentPlan TEXT, is_TreatmentPlanGood TEXT, is_TreatmentPlanOverdue TEXT, Admitted TEXT'

In [37]:
table_name= "ptTreatmentPlanDuenessSummaryTemp"
try:
    c = conn.cursor()
    c.execute(f'''CREATE TABLE IF NOT EXISTS {table_name} ({query}) ''')
    conn.commit()
    allPatientDf.to_sql("ptTreatmentPlanDuenessSummaryTemp", conn, if_exists='replace', index = False)
#     logging.info(f"Successfully pushed {table_name} to database.")            
except Exception as e:
#     logging.error(f"Failed to push {table_name} to database.", exc_info=True) 
    print(e)

In [24]:
conn.close()

## Old Code

In [25]:
# final_df[final_df['ProgramName'] =='Boro Park Clinics'].head(15).to_excel("../data/checkTreatmentPlanCode.xlsx", index=False)

In [26]:
# final_df[final_df['Patient Name'] == 'Braun, Chesky']

In [27]:
# final_df['Program']
# final_df['Status']
# final_df = final_df.merge(patientStatusProgramdf, on='MRN', how='inner')
# final_df

In [28]:
# final_df.columns

In [29]:
# final_df = final_df[['MRN', 'Patient Name_x', 'Encounter Type',
#                    'Encounter / Service Provider', "Encounter Facility", 'Visit Date', 'Signed By',
#                    'Signed Date', 'Co Sign Status', 'Co Signed By',
#                    'Summarized Encounter Type', 'Missing Initial Treatment Plan',
#                    'Review Date', 'Days to Review Date', 'allTheInfoYouEverWanted','Patient Status',
#                    'Date of Last Appointment', 'Total Appointments', 'Updated Status_x', "Last Seen_x"]].copy()

In [30]:
# writer = pd.ExcelWriter("Excel TreatmentPlan Data/TreatmentPlanforCompliance v5.xlsx", engine='xlsxwriter')

# # Convert the dataframe to an XlsxWriter Excel object. Turn off the default
# # header and index and skip one row to allow us to insert a user defined
# # header.
# final_df.to_excel(writer, sheet_name='Sheet1', startrow=1, header=False, index=False)

# # Get the xlsxwriter workbook and worksheet objects.
# workbook = writer.book
# worksheet = writer.sheets['Sheet1']

# # Get the dimensions of the dataframe.
# (max_row, max_col) = final_df.shape

# # Create a list of column headers, to use in add_table().
# column_settings = []
# for header in final_df.columns:
#     column_settings.append({'header': header})

# # Add the table.
# worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})

# # Make the columns wider for clarity.
# worksheet.set_column(0, max_col - 1, 12)

# # Close the Pandas Excel writer and output the Excel file.
# writer.save()
# writer.close()